In [0]:
#%pip install google_play_scraper

In [0]:
from google_play_scraper import Sort, reviews_all

# app id for each delivery app 
app_ids = {
    "coupangEats" : "com.coupang.mobile.eats",
    "baemin" : "com.sampleapp",
    "yogiyo" : "com.fineapp.yogiyo"
}

# save app names
app_names = [key for key in app_ids.keys()]

1. 초기 접근법: 다국가 리뷰 수집 시도 <br/>
프로젝트 초기에는 외국인 리뷰를 폭넓게 확보하기 위해, 한국과 교류가 많거나 주요 언어를 사용하는 여러 국가의 리뷰를 동시에 수집하는 전략을 세웠습니다. 아래 코드는 당시 최초로 설계했던 데이터 수집 로직입니다.

In [0]:
# ------------ This code is no longer in use -----------------

# countries to check reviews
# I am covering countries:
# 1) using English, French, Spanish, and Chinese as official or major secondary languages
# 2) relatively nearby Korea (with an assumption there will be more tourists coming from these countries due to its accessibility)
# 3) with high exchange scope
# 4) and Korea
language_group = [
    # English
    'us', 'gb', 'ca', 'au', 'nz', 'ie', 'za', 'in', 'sg', 'ph', 
    # Spanish 
    'es', 'mx', 'ar', 'co', 'cl', 'pe',  
    # Chinese                        
    'cn', 'tw', 'hk',   
    # French                                         
    'fr', 'be', 'ch', 'sn'                                       
]
nearby_group = ['jp', 'ru', 'mn', 'vn', 'id', 'th']
exchange_scope_group = ['au', 'sa', 'de', 'ae']
korea_group = ['kr']

# combine countries
country_codes = language_group + nearby_group + exchange_scope_group + korea_group

# remove duplicates
unique_country_codes = list(set(country_codes))

# sort options to iterate to get more reviews
sort_options = [Sort.NEWEST, Sort.MOST_RELEVANT, Sort.RATING]

all_reviews = []
# get reviews from each country in each delivery app 
for app_name, app_id in app_ids.items():
    for country_code in unique_country_codes:
        # reviews = reviews_all(
        #     app_id,
        #     sleep_milliseconds = 500, # Add a delay to avoid rate-limiting
        #     lang = 'en', 
        #     country = country_code,
        #     sort = Sort.NEWEST, 
        # )

        # for review in reviews:
        #     review['appName'] = app_name
        #     review['appId'] = app_id
        #     review['countryCode'] = country_code

        # all_reviews.extend(reviews)
        # print(f"Found {len(reviews)} reviews for {app_name} in {country_code}")
        pass
print("위 코드를 실행한 결과, 모든 국가에서 동일한 리뷰가 수집되는 문제를 발견했습니다.")
    


2. 전략 수정 <br/>
위 코드로 데이터를 수집한 결과, 모든 국가에서 동일한 개수의 리뷰가 반환되는 문제를 발견했습니다. 이는 대상 앱들이 한국 내수용이라 구글 플레이스토어가 미출시 국가에 대해 한국 스토어의 데이터를 기본값으로 보여주기 때문이었습니다. <br/>
따라서 비효율적인 다국가 수집 방식을 폐기하고, 목표에 더 직접적으로 접근할 수 있는 새로운 전략을 채택했습니다.

In [0]:
import pandas as pd

# all_reviews 리스트를 데이터프레임으로 변환했다고 가정
df = pd.DataFrame(all_reviews)

# '요기요' 앱의 독일(de) 리뷰와 일본(jp) 리뷰를 각각 5개씩 가져오기
yogiyo_de_reviews = df[(df['appName'] == 'coupangEats') & (df['countryCode'] == 'de')].head()
yogiyo_jp_reviews = df[(df['appName'] == 'coupangEats') & (df['countryCode'] == 'jp')].head()

# 두 데이터프레임의 내용 (userName, content, at 등)이 완전히 똑같은지 확인
print("--- Yogiyo Reviews from Germany ---")
print(yogiyo_de_reviews[['userName', 'content', 'at']])

print("\n--- Yogiyo Reviews from Japan ---")
print(yogiyo_jp_reviews[['userName', 'content', 'at']])

In [0]:
from google_play_scraper import Sort, reviews_all
import pandas as pd

app_id = 'com.sampleapp' # Replace with the actual app ID

us_reviews = reviews_all(
    app_id,
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='us', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

#df_reviews = pd.DataFrame(us_reviews)
#print(df_reviews.head())

In [0]:
# filtering on language written on the review: for further function, 
# I should filter out korean ones for simplicity


# from google_play_scraper import Sort, reviews_all
# import pandas as pd
# from langdetect import detect, LangDetectException

# app_id = 'com.busuu.android.en'

# # 1. Scrape all reviews as before
# print("Fetching all reviews...")
# all_us_reviews = reviews_all(
#     app_id,
#     sleep_milliseconds=0,
#     lang='en',
#     country='us',
#     sort=Sort.NEWEST,
# )

# df = pd.DataFrame(all_us_reviews)
# print(f"Successfully fetched {len(df)} total reviews.")

# # 2. Detect the language for each review
# print("Detecting language for each review...")
# detected_langs = []
# for review_text in df['content']:
#     try:
#         # Detect the language of the review content
#         lang = detect(review_text)
#         detected_langs.append(lang)
#     except LangDetectException:
#         # If detection fails (e.g., for short or non-text reviews), mark as 'unknown'
#         detected_langs.append('unknown')

# # Add the detected languages as a new column
# df['detected_language'] = detected_langs

# # 3. Filter the DataFrame for English reviews
# df_english = df[df['detected_language'] == 'en'].copy()

# print(f"\nFound {len(df_english)} reviews written in English.")
# print(df_english[['content', 'score', 'detected_language']].head())